# Description

Perform asset optimization 


Methods : 
* Equally weighted
* 
* 
* 

# Setup

In [1]:
%matplotlib notebook
%load_ext autoreload

## Imports

In [2]:
# os related
from pathlib import Path
import os 

# data related
import pandas as pd
import missingno as msno
import datetime as dt
import numpy as np


# visual related
import matplotlib.pyplot as plt
# time related

In [3]:

from fonctions import *

## Paths

In [4]:
# MUST BE IN CODE FOLDER TO WORK !
PROJECT_PATH = Path(os.getcwd()).parent

RAW_DATA = PROJECT_PATH / "RAW_DATA" 
PROC_DATA = PROJECT_PATH / "PROC_DATA"

# RAW FILES
CSV_FILE = RAW_DATA / "Individual_stock_data_csv.csv"
XLSX_FILE = RAW_DATA / "Individual_stock_data.xlsx"

# PROC FILES
PROC_STOCK = PROC_DATA / "proc_stock.csv"
PROC_CARBON = PROC_DATA / "proc_carbon.csv"

STOCK_2000 = PROC_DATA / "stock_2000.csv"


STOCK_WORK = PROC_DATA / "stock_work.csv"
STOCK2USE = PROC_DATA / "stock2use.csv"
DIC2USE = PROC_DATA / "dic2use.json"

## Utils

# Notebook

## Loading data

In [5]:
df = read_file(STOCK2USE)

In [7]:
mean, std = equaly_weighted(df)

In [16]:
df.cov

<bound method DataFrame.cov of                 aapl      intu      klac      nvda      adsk        has  \
timestamp                                                                 
1999-12-14    723.14   1052.36   3570.27    188.89   1732.45   12798.09   
1999-12-21    781.26   1041.73   4664.87    215.88   1796.35   11551.52   
1999-12-28    748.39   1109.05   4691.89    226.04   1807.00   12091.70   
2000-01-04    781.26   1150.39   4491.89    231.43   1803.45   11468.42   
2000-01-11    706.94   1556.69   4997.30    210.80   1710.92   11219.11   
...              ...       ...       ...       ...       ...        ...   
2019-11-12  64259.93  10705.25  26537.44  13861.57  37195.60   98409.38   
2019-11-19  65322.11  11210.91  26904.27  13754.43  38718.16   99295.50   
2019-11-26  64831.50  10948.20  24785.44  14350.27  40193.65  104705.90   
2019-12-03  63644.21  10445.84  23906.42  13742.68  41276.14  103444.50   
2019-12-10  65859.31  10389.84  24822.13  14160.93  41996.25  104174.